In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Previous version
class Pedro_Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None


class Pedro_Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Pedro_Variable(y)
        self.input = input
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

class Pedro_Square(Pedro_Function):
    def forward(self, x):
        return x ** 2

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

class Pedro_Exp(Pedro_Function):
    def forward(self, x):
        return np.exp(x)

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx


def numerical_diff(f, x, eps=1e-4):
    x0 = Pedro_Variable(x.data - eps)
    x1 = Pedro_Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

In [3]:
# New version - update of Pedro_Variable & Pedro_Function
class Pedro_Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func


class Pedro_Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Pedro_Variable(y)
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

class Pedro_Square(Pedro_Function):
    def forward(self, x):
        return x ** 2

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

class Pedro_Exp(Pedro_Function):
    def forward(self, x):
        return np.exp(x)

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx


def numerical_diff(f, x, eps=1e-4):
    x0 = Pedro_Variable(x.data - eps)
    x1 = Pedro_Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

In [4]:
# manual forward propagation
A = Pedro_Square()
B = Pedro_Exp()
C = Pedro_Square()

x = Pedro_Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

In [5]:
# manual backward propagation
y.grad = np.array(1.0)

C = y.creator
b = C.input
b.grad = C.backward(y.grad)

B = b.creator
a = B.input
a.grad = B.backward(b.grad)

A = a.creator
x = A.input
x.grad = A.backward(a.grad)

print(x.grad)

3.297442541400256


In [6]:
# Auto backward propagation
class Pedro_Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        f = self.creator
        if f is not None:
            x = f.input
            x.grad = f.backward(self.grad)
            x.backward()

In [7]:
# Auto backward propagation
A = Pedro_Square()
B = Pedro_Exp()
C = Pedro_Square()

x = Pedro_Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256
